In [2]:
################ importing libraries ###############
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [3]:
hul = pd.read_excel("C:/Users/10586/Downloads/USA BGS Data_Pivot Table.xlsx")

In [4]:
miss_perc=hul.isna().sum()/len(hul)*100
miss_perc


Period                                                 0.000000
Brand                                                  0.000000
Power (Mean)                                           0.000000
Salience (Mean)                                        0.000000
Would you recommend to others                         14.285714
                                                        ...    
On a coupon                                            0.030525
Inspires women to express their beauty confidently    42.521368
Are recommended by dermatologists/doctors              0.061050
Fulfilled (Mean)                                       0.000000
Premium (Mean)                                         0.000000
Length: 81, dtype: float64

In [5]:
#removing rows where Brand power id missing since that is our dependent variable
#or can we take true singnal forecast where it's missing?
hul_BP=hul[hul['Power (Mean)'].notna()]


In [8]:
#variables removed due to high missing
hul_BP_RemoveAttributesWithHighMiss=hul_BP.dropna(thresh=hul_BP.shape[0]*0.6, how='all',axis=1)
miss_perc_BP_RHM=hul_BP_RemoveAttributesWithHighMiss.isna().sum()/len(hul_BP_RemoveAttributesWithHighMiss)*100
miss_perc_BP_RHM

Period                                        0.000000
Brand                                         0.000000
Power (Mean)                                  0.000000
Salience (Mean)                               0.000000
Would you recommend to others                14.285714
                                               ...    
Taste/Work better than other brands          14.285714
On a coupon                                   0.030525
Are recommended by dermatologists/doctors     0.061050
Fulfilled (Mean)                              0.000000
Premium (Mean)                                0.000000
Length: 69, dtype: float64

In [9]:
maxValues = hul_BP_RemoveAttributesWithHighMiss.max()
 
print(maxValues)

Period                                       2022-04-03 00:00:00
Brand                                                       Tone
Power (Mean)                                           38.286221
Salience (Mean)                                       492.763465
Would you recommend to others                             99.999
                                                    ...         
Taste/Work better than other brands                       99.999
On a coupon                                               99.999
Are recommended by dermatologists/doctors                 99.999
Fulfilled (Mean)                                        0.350966
Premium (Mean)                                          4.161714
Length: 69, dtype: object


In [10]:
hul_BP_RemoveAttributesWithHighMiss.head(5)

,Period,Brand,Power (Mean),Salience (Mean),Would you recommend to others,Top of mind awareness,Transforms how you feel,Bought last,Leaves you feeling more refreshed,MeetNeeds (Mean),...,Work as well as more expensive brands,Provide the right amount of lather,Lasts longer than other brands,Are too cheap to be of acceptable quality,Are more natural than other brands,Taste/Work better than other brands,On a coupon,Are recommended by dermatologists/doctors,Fulfilled (Mean),Premium (Mean)
0,2017-01-15,Aveeno,6.326076,93.463514,25.00,4.000,25.00,7.690,20.83,6.200000,...,4.17,25.00,25.00,4.170,41.67,16.67,24.00,37.50,0.038151,1.579868
1,2017-01-22,Aveeno,7.298652,55.883676,14.71,1.450,13.24,0.001,14.71,7.115385,...,10.29,13.24,5.88,1.470,29.41,14.71,8.70,25.00,0.042458,1.846455
2,2017-01-29,Aveeno,5.394297,66.624500,17.11,1.280,13.16,2.440,13.16,6.814815,...,7.89,18.42,9.21,1.320,27.63,10.53,8.97,23.68,0.028986,1.606171
3,2017-02-05,Aveeno,7.453036,74.857497,16.36,0.001,9.09,0.001,18.18,6.045455,...,14.55,18.18,10.91,0.001,25.45,18.18,3.57,32.73,0.041632,1.342316
4,2017-02-12,Aveeno,6.509035,87.417365,25.00,6.820,10.00,0.001,20.00,6.176471,...,10.00,10.00,5.00,0.001,15.00,25.00,4.55,20.00,0.040536,0.980532


In [11]:
miss_perc_BP_RHM #final variableshaving less than 60% of missing

Period                                        0.000000
Brand                                         0.000000
Power (Mean)                                  0.000000
Salience (Mean)                               0.000000
Would you recommend to others                14.285714
                                               ...    
Taste/Work better than other brands          14.285714
On a coupon                                   0.030525
Are recommended by dermatologists/doctors     0.061050
Fulfilled (Mean)                              0.000000
Premium (Mean)                                0.000000
Length: 69, dtype: float64

In [10]:
cor_mat=hul_BP_RemoveAttributesWithHighMiss.corr() # corr among the attributes
cor_mat

,Power (Mean),Top of mind awareness,Inspire women to feel more positive about the way they look,Occasional User,Moisturise your skin better than other brands,Premium (Mean),Rejector,Potential (Mean),Provide the right amount of lather,Protect effectively from germs,...,Make you feel pampered,Are too cheap to be of acceptable quality,Affinity (Mean),Effectively remove body odour,Are more natural than other brands,Meaning (Mean),Effectively make your skin fairer/lighter,Do people talk a lot about,Aware,Somewhere Else
Power (Mean),1.000000,0.928994,0.555435,0.933303,0.633679,0.806027,-0.827777,0.532932,0.899359,0.749403,...,0.663846,-0.167710,0.854410,0.850161,0.862534,0.904873,0.363261,0.897488,0.844871,0.646646
Top of mind awareness,0.928994,1.000000,0.381461,0.908365,0.467266,0.638769,-0.696896,0.539729,0.787560,0.669440,...,0.509538,-0.092551,0.717355,0.731119,0.745432,0.770265,0.194211,0.783126,0.702707,0.548144
Inspire women to feel more positive about the way they look,0.555435,0.381461,1.000000,0.462930,0.905925,0.654932,-0.627945,0.102828,0.661976,0.127401,...,0.891899,-0.324661,0.633893,0.434214,0.600149,0.483961,0.884029,0.627236,0.617704,0.429058
Occasional User,0.933303,0.908365,0.462930,1.000000,0.536854,0.699150,-0.800191,0.469860,0.854484,0.697182,...,0.591539,-0.063860,0.766359,0.782176,0.804200,0.840837,0.279896,0.848404,0.763650,0.591525
Moisturise your skin better than other brands,0.633679,0.467266,0.905925,0.536854,1.000000,0.731758,-0.670704,0.188259,0.736436,0.278430,...,0.920706,-0.357841,0.716522,0.557342,0.697076,0.564046,0.811661,0.715435,0.678108,0.479006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Meaning (Mean),0.904873,0.770265,0.483961,0.840837,0.564046,0.795140,-0.721641,0.575478,0.830420,0.717308,...,0.597678,-0.257443,0.852303,0.797500,0.799467,1.000000,0.317300,0.824251,0.773458,0.588283
Effectively make your skin fairer/lighter,0.363261,0.194211,0.884029,0.279896,0.811661,0.485776,-0.453282,0.006528,0.458900,-0.073889,...,0.803604,-0.307267,0.464310,0.234512,0.440197,0.317300,1.000000,0.430763,0.430846,0.294738
Do people talk a lot about,0.897488,0.783126,0.627236,0.848404,0.715435,0.828245,-0.863467,0.453905,0.928875,0.725080,...,0.752303,-0.222505,0.860435,0.863861,0.886271,0.824251,0.430763,1.000000,0.874267,0.663894
Aware,0.844871,0.702707,0.617704,0.763650,0.678108,0.826379,-0.805734,0.418436,0.859170,0.699870,...,0.693331,-0.240042,0.828208,0.826209,0.824430,0.773458,0.430846,0.874267,1.000000,0.756232


In [12]:
cor_mat=cor_mat[['Power (Mean)']] # corr with Brand Power
cor_mat

NameError: name 'cor_mat' is not defined

In [12]:
high_corr=cor_mat[abs(cor_mat['Power (Mean)'])>0.8] #------- these variables can go into modelling
low_corr=cor_mat[abs(cor_mat['Power (Mean)'])<0.2]
high_corr

,Power (Mean)
Power (Mean),1.000000
Top of mind awareness,0.928994
Occasional User,0.933303
Premium (Mean),0.806027
Rejector,-0.827777
Provide the right amount of lather,0.899359
Incidental User,0.827876
Are enjoyable to use,0.915449
Consideration (Mean),0.901893
Transforms how you feel,0.874639


In [13]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [17]:
X = hul_BP_RemoveAttributesWithHighMiss.iloc[:,3:]
X.dropna(inplace=True)
vif_mat=calc_vif(X) #INF attributes are a linear combo of other variables. remove them

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [15]:
# Y=X.drop(['Rejector','Incidental User'],axis=1)
Y=X.drop(['TBCA','Total brand awareness','Aided brand awareness','Not aware','Aware'],axis=1)
vif_mat=calc_vif(Y)

# ,'TBCA','Total brand awareness','Aided brand awareness','Not aware','Aware'
#seems most of thyem are correlated. go to bayesian modelling

In [16]:
vif_mat.sort_values(by='VIF', ascending = False)

,variables,VIF
45,Consumer Loyalty (Mean),2796.890656
26,Fulfilled (Mean),1503.582787
9,Incidental User,1098.644608
5,Rejector,1044.140834
29,Converted (Mean),255.350156
0,Top of mind awareness,253.688906
4,Premium (Mean),162.922176
35,Salience (Mean),116.450757
8,Protect effectively from germs,42.816141
23,Regular User,40.416706
